# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [ ]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

#### Helper functions

In [ ]:
help(fd)

#### Generate a list of filenames

In [ ]:
fish_filenames = fd.generate_filenames_list('data/train/', subfolders = True)
print("There are {} filenames in the master set list".format(len(fish_filenames)))
test_filenames = fd.generate_filenames_list('data/test_stg1/', subfolders = False)
print("There are {} filenames in the test set list".format(len(test_filenames)))

#### Generate the labels for the master set list

In [ ]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
print("One label per row entry: {}".format(all(np.sum(fish_label_arr, 1) == 1) ))

#### Shuffle and split the master set list into training and validation sets

In [ ]:
valid_size = 200
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print("Validation set size: {}".format(y_val.shape[0]))
print("Training set size: {}".format(y_train.shape[0]))

## Graph and Session Runs

#### Graph parameters

In [ ]:
num_epochs = 30
#Preprocessing
std_y = 300
std_x = 500

# General
num_channels = 3
num_labels = 8
batch_size = 25
stddev = 0.2

# convolution
kernel_sizes = [3, 3, 3, 3, 3, 3]
conv_depths = [64, 128, 256, 512, 256, 128] # NOTE : 64 currently not used with dilation2d
conv_strides = [4, 1, 1, 1, 1, 1]

pool_strides = [2, 2, 2, 2]

final_depth = conv_depths[-1]

#dropout
kp = 0.75

# fully connected
fc1_depth = 256
fc2_depth = 64

#regularization
beta = 1e-1 

# Learning rate
init_rate = 5e-4
per_steps = len(files_train)
decay_rate = 0.9

# Report rate
validate_interval = 30


#### Session parameters

In [ ]:
version_ID = 'v1.3.3.2'
logs_path = os.getcwd()+'/TB_logs/'+version_ID

In [ ]:
%run -i 'GRAPH.py'

In [ ]:
%run -i 'SESSION.py'